In [3]:
import sys
print(sys.executable)

C:\Users\User\Anaconda3\envs\geePy3\python.exe


In [4]:
import ee
import geemap
import os
import geopandas as gpd
import pandas as pd
import shutil

# Intermediate results can be shown on the map

In [5]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# From a gee feature (point) to a Rectangle ROI (in UTM)

In [6]:
def create_roi(feature):
	buffer = feature.transform(ee.Projection(feature.get("EPSG")), ee.ErrorMargin(1, "meters")).buffer(400)
	#// Get bounding boxx and simplify it to approximate a North aligned square ROI
	return buffer.bounds().simplify(ee.ErrorMargin(1, "meters"))

def set_utm_epsg(feature):
	coords = feature.geometry().coordinates()
	lon = ee.Number(coords.get(0))
	lat = ee.Number(coords.get(1))
	epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
	#print(epsg.getInfo())
	return feature.set("EPSG",ee.String("EPSG:").cat(ee.String(str(epsg.getInfo()))))#ee.String("EPSG:").cat

def add_xy(img):
	return img.addBands(ee.Image.pixelLonLat().rename(["X", "Y"]))

- **.getInfo()** (e.g., in Line 12 of the above cell) is alway necessary, the same when we want to print sth, which is not the case when using GEE directly

- get image patch for one gee feature (point)

# From a gee feature to a good s1/2 patch

In [7]:
def get_s1_mosaic(city):
    #input is a gee feature
    
    roi = city.geometry()

    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(timePeriod).filterBounds(roi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(s1_bands) \
    .map(clip_roi) \
    .median()
    return collection

#this is just a simple example to get cloud-less s2
def get_s2_cloud_free(city):
    #input is a gee feature
    
    roi = city.geometry()
    
    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate(timePeriod) \
    .filterBounds(roi) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
    .select(s2_bands) \
    .map(clip_roi) \
    .mosaic()
    return collection

# Download image patches

## Set path and para.

In [25]:
top_folder = r".\files4shTest\geojson"

out_dir = r".\files\res"

timePeriod = ee.DateRange('2016-01-01', '2017-01-01')

start_date = ee.Date.fromYMD(2016, 1, 1)
end_date = ee.Date.fromYMD(2017, 1, 1)
    
s2_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s1_bands = ["VV", "VH"]

## Main steps:

- loop all the cities
- loop all the points in one city: convert each point to a gee feature
- find the image and download
- record number of points per class per city

## read a geojson file

In [31]:
import json

roiFile=os.path.join(top_folder, "Rio_de_Janeiro"+".geojson")


with open(roiFile) as f:
    data = json.load(f)

#print(data['features'])

print(len(data['features']), data['features'][1])
for feature in data['features']:

    cor=feature['geometry']['coordinates']
    #print(cor)

427 {'type': 'Feature', 'properties': {'Label': 1.0}, 'geometry': {'type': 'Point', 'coordinates': [-43.113008962198165, -22.902129663498574]}}


## loop the geojson files and download data for each point

In [30]:
from geeCode_qgis import exportCloudFreeSen2

id = 0
for path, dirs, files in os.walk(top_folder):


    #print(num_label)
    print('!!!!!!!!!!!!!!!!!', path, dirs, files)

    for d in files:
        #sh_path = os.path.join(top_folder, d)
        print(d)

        file=os.path.join(top_folder, d+".geojson")
        with open(file) as f:
            data = json.load(f)
            
        print(file)

        num_label = pd.Series(0, index=range(1, 18))

        'prepare a empty city folder to save download images'
        cityFolder = os.path.join(out_dir, d)
        if os.path.exists(cityFolder):
            shutil.rmtree(cityFolder, ignore_errors=True)

        'download a patch for each point'
        for i in [19]:#range(len(data['features'])):#shapefile.shape[0]

            'convert one row in the shapefile into a feature for gee'
            features = []

            #geom = shapefile.iloc[i:i + 1, :]
            #print('geom',geom)

            #'''convert to gee feature'''
            #jsonDict = eval(geom.to_json())
            #geojsonDict = jsonDict['features'][0]
            feature=ee.Feature(data['features'][i])
            #
            print(feature.getInfo())

            #Get the individual geometries as a list.
            geometries = feature.geometry()
            #print(geometries.getInfo())

            pointsC = geometries.coordinates()
            print(pointsC.getInfo())

            label = feature.get("Label").getInfo()

            print(label, pointsC.size().getInfo())
            #print('Point 1', geometries.get(0))

            '''record the num of labels'''
            num_label[label]=num_label[label]+1

            'set path to save the download images'
            cityFolder_label = os.path.join(cityFolder, str(label))
            if not os.path.exists(cityFolder_label):
                    os.makedirs(cityFolder_label)
            filename = os.path.join(cityFolder_label, str(num_label[label])+'.tif')

            #print(feature.getInfo())
            feature_epsg = set_utm_epsg(feature)
            #print(feature_epsg)
            roiF = create_roi(feature_epsg)

            'find img'
            #this is a negative example of cloud-less images
            s2 = get_s2_cloud_free(roiF)
            
            s2_masked = exportCloudFreeSen2(roiF.geometry(), start_date, end_date)
            s2_masked = s2_masked.select(s2_bands)
            
            
            s1 = get_s1_mosaic(roiF)
            
            'for debug, but the visualization is not good, maybe due to the small size of roi'
            #print(s2.getInfo())
            Map.addLayer(s2_masked, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2_m")
            
            Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2")
            #Map.addLayer(s1, {'bands': ['VV', 'VH', 'VV'], 'min': -30, 'max': 0}, "S1", False)
            #print(roiF.geometry().getInfo())
            Map.addLayer(roiF, {}, 'roiF')
            Map.centerObject(feature.geometry(), 15)
            Map

            'download'
            image = s2_masked.addBands(s1)# s2.addBands(s1)
            
            roi = ee.Geometry(roiF.geometry())
            #print(roi.getInfo())
            roi = roi.transform(feature_epsg.get("EPSG"), ee.ErrorMargin(1, "meters"))
            print(roi.getInfo())
            #the transformation here is to make sure that the downloaded images has the expected size (decided with buffer in function create_roi)
            
            print(feature_epsg.get("EPSG").getInfo())
            geemap.ee_export_image(image, filename=filename, scale=10, region=roi, crs=ee.Projection(feature_epsg.get("EPSG")), file_per_band=False)#, region=roi

        if id == 0:
            label_per_City = pd.DataFrame(num_label,columns=[d])
        else:
            label_per_City[d] = num_label
        id = 1

        print(label_per_City)

!!!!!!!!!!!!!!!!! .\files\geojson [] ['Rio_de_Janeiro.geojson']
Rio_de_Janeiro.geojson
.\files\geojson\Rio_de_Janeiro.geojson.geojson
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-43.23256744314287, -22.927887237130054]}, 'properties': {'Label': 1}}
[-43.23256744314287, -22.927887237130054]
1 2
{'geodesic': False, 'crs': {'type': 'name', 'properties': {'name': 'EPSG:32723'}}, 'type': 'Polygon', 'coordinates': [[[680843.0261719562, 7462980.112749467], [681639.4950473332, 7462970.536778068], [681649.0975748855, 7463767.580970694], [680852.5865477637, 7463777.1546052005], [680843.0261719562, 7462980.112749467]]]}
EPSG:32723
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'E:\\0Learn\\0LearningNotes_python_geo_rs\\files\\res\\Rio_de_Janeiro.geojson\\1\\1.zip'
    Rio_de_Janeiro.geojson
1                        1
2                        0
3                        0
4                       

## Important notes:

- be careful of data CRS
- display/print interm results on map to avoid mistakes
- unsure code can be first tested with GEE: https://code.earthengine.google.com
- useful Ref: https://github.com/giswqs/earthengine-py-notebooks

# Question

- 